code by Jakob

In [1]:
import pandas as pd
import logging

In [2]:
log = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

In [3]:
data_dir = "data/"
dataframe_dir = data_dir + "dataframes/"
speeches_dir = data_dir + "speeches/"
speeches_pdf_dir = speeches_dir + "pdf/"
speeches_txt_dir = speeches_dir + "txt/"
speeches_extracted_dir = speeches_dir + "extracted/"
last_metadata_step = 'metadata_s02'
current_metadata_step = 'metadata_s03'

speech_suffix = "speeches.txt"
extracted_speech_suffix = lambda x,numb=True: f"speech_{x}.txt" if numb else "speech.txt"

## load data

In [4]:
# sr = pd.read_csv(f'{dataframe_dir}/speech_records.csv')
metadata = pd.read_csv(f'{dataframe_dir}/{last_metadata_step}.csv')
dpi = pd.read_csv(f'{dataframe_dir}/documents_processing_info.csv')

fill up missing values and convert dates

In [5]:
metadata.date = pd.to_datetime(metadata.date)
dpi.day = dpi.day.fillna(1)
dpi.month = dpi.month.fillna(1)
dpi['date'] = pd.to_datetime(dpi[['year','month','day']])
dpi.drop(columns=['year','month','day'], inplace=True)

## Proprocessing
Check which pdfs are image and which are text based - we did that by introducing documents_processing_info where we store the classficiation of documents, new we map this info back into the metadata. 
Therefore we create sub dfs for mapping once with and without sub_body to my documents_processing_info to obtain the text_based column

In [6]:
dpi_sb = dpi[dpi["sub_body"].notna()].copy()
dpi_nsb = dpi[dpi["sub_body"].isna()].copy()
dpi_sb = dpi_sb.sort_values(["date", "body", "sub_body"])
dpi_nsb = dpi_nsb.sort_values(["date", "body"])

dpi_listed_subbodies = dpi[dpi.sub_body.notna()].sub_body.unique()
md_sb = metadata[(metadata.sub_body.isin(dpi_listed_subbodies))].copy()
md_nsb = metadata[~(metadata.sub_body.isin(dpi_listed_subbodies))].copy()
md_sb = md_sb.sort_values(["date", "body", "sub_body"])
md_nsb = md_nsb.sort_values(["date", "body"])

In [7]:
md_merg_sb = pd.merge_asof(
    md_sb,
    dpi_sb,
    by=["body", "sub_body"],
    left_on="date",
    right_on="date",
    direction="backward",
    suffixes=("", "_todrop")
)
md_merg_nsb = pd.merge_asof(
    md_nsb,
    dpi_nsb,
    by=["body"],
    left_on="date",
    right_on="date",
    direction="backward",
    suffixes=("", "_todrop")
)

In [8]:
metadata = pd.concat([md_merg_sb, md_merg_nsb]).sort_values(["body", "date"])
metadata.drop(columns=['sub_body_todrop'], inplace=True)

notable exceptions

In [9]:
metadata.loc[metadata.document_symbol=='E/2000/SR.44', 'text_based'] = False

In [ ]:
metadata.groupby(['body', 'sub_body', 'text_based'], dropna=False).first()

record_id                                speaker  \
body sub_body text_based                                                     
A    C.1      False          288722  Pérez de Cuéllar, Javier, 1920-2020   
              True           365914      Boutros-Ghali, Boutros, 1922-2016   
     C.2      False          328834  Pérez de Cuéllar, Javier, 1920-2020   
              True           393116                 Annan, Kofi, 1938-2018   
     C.5      False          292642  Pérez de Cuéllar, Javier, 1920-2020   
              True           356413      Boutros-Ghali, Boutros, 1922-2016   
     ES-10    True           428414                 Annan, Kofi, 1938-2018   
     ES-11    True          3967632              Guterres, António, 1949-   
     S-13     False          311611  Pérez de Cuéllar, Javier, 1920-2020   
     S-14     False          312127  Pérez de Cuéllar, Javier, 1920-2020   
     S-15     False          326292  Pérez de Cuéllar, Javier, 1920-2020   
     S-16     False          338057  Pérez de Cuéllar, Javier, 1920-2020   
     S-17     False          338854  Pérez de Cuéllar, Javier, 1920-2020   
     S-18     False          339005  Pérez de Cuéllar, Javier, 1920-2020   
     S-19     True           385917                 Annan, Kofi, 1938-2018   
     S-20     True           395693                 Annan, Kofi, 1938-2018   
     S-21     True           401981                 Annan, Kofi, 1938-2018   
     S-23     True           426737                 Annan, Kofi, 1938-2018   
     S-24     True           426121                 Annan, Kofi, 1938-2018   
     S-25     True           445545                 Annan, Kofi, 1938-2018   
     S-26     True           448429                 Annan, Kofi, 1938-2018   
     S-27     True           472002                 Annan, Kofi, 1938-2018   
     S-28     True           544640                 Annan, Kofi, 1938-2018   
     S-29     True           783952                    Ban, Ki-moon, 1944-   
     S-31     True          3922120              Guterres, António, 1949-   
     NaN      False          380162                 Lie, Trygve, 1896-1968   
              True           363235      Boutros-Ghali, Boutros, 1922-2016   
E    NaN      False         3984934                 Lie, Trygve, 1896-1968   
              True           368394      Boutros-Ghali, Boutros, 1922-2016   
S    NaN      False         1317185                    Thant, U, 1909-1974   
              True           370162      Boutros-Ghali, Boutros, 1922-2016   

                           speaker_organization document_symbol       date  \
body sub_body text_based                                                     
A    C.1      False       UN. Secretary-General  A/C.1/38/PV.11 1983-10-24   
              True        UN. Secretary-General   A/C.1/49/PV.3 1994-10-17   
     C.2      False       UN. Secretary-General  A/C.2/43/SR.16 1988-10-19   
              True        UN. Secretary-General  A/C.2/52/SR.45 1997-12-01   
     C.5      False       UN. Secretary-General   A/C.5/38/SR.7 1983-10-07   
              True        UN. Secretary-General  A/C.5/47/SR.71 1993-08-26   
     ES-10    True        UN. Secretary-General   A/ES-10/PV.14 2000-10-20   
     ES-11    True        UN. Secretary-General    A/ES-11/PV.1 2022-02-28   
     S-13     False       UN. Secretary-General     A/S-13/PV.1 1986-05-27   
     S-14     False       UN. Secretary-General     A/S-14/PV.1 1986-09-17   
     S-15     False       UN. Secretary-General     A/S-15/PV.1 1988-05-31   
     S-16     False       UN. Secretary-General     A/S-16/PV.1 1989-12-12   
     S-17     False       UN. Secretary-General     A/S-17/PV.1 1990-02-20   
     S-18     False       UN. Secretary-General     A/S-18/PV.1 1990-04-23   
     S-19     True        UN. Secretary-General     A/S-19/PV.1 1997-06-23   
     S-20     True        UN. Secretary-General     A/S-20/PV.1 1998-06-08   
     S-21     True        UN. Secretary-General

# Filter the Summary Records
The dataset contains a lot of summary records, we decided to filter them out at this point since the speeches do not resemble the actual words used and we want to extract from here on only the speeches and not the general agenda of things talked about. But we kept them in till here so we already have the pdfs if the research question gets broader in the future

In [ ]:
sr_speeches = metadata[metadata.document_symbol_found.str.contains('/SR.')]
print(f'there are {len(sr_speeches)} speeches that are a summary record')

there are 182 speeches that are a summary record


In [ ]:
metadata = metadata[~metadata.document_symbol_found.str.contains('SR')]

In [ ]:
metadata.reset_index(drop=True, inplace=True)
metadata.to_csv(f'{dataframe_dir}{current_metadata_step}.csv', index=False)

### quick info

In [ ]:
metadata.text_based.value_counts()

text_based
True     725
False    285
Name: count, dtype: int64